In [ ]:
# To enable faster auto-complete
%config Completer.use_jedi = False

In [ ]:
import sys 
import os
import tensorflow as tf
from tensorflow import keras
from utils import energy_model, data_augmentation
import numpy as np
import re
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix


In [ ]:
# Selecting the GPU to be used 
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    # Restrict tensor flow to use GPU-1
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Set GPUs before initializing
        print(e)

In [ ]:
def confusion_matrix(model, X_test, y_test, class_names):
    # Prediction on test data set -> predicting classes
    y_pred = model.predict_classes(X_test)
    con_mat = tf.math.confusion_matrix(labels=y_test, predictions=y_pred).numpy()
    # Normalizing the confusion matrix
    con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1) [:, np.newaxis], decimals=2)
    con_mat_df = pd.DataFrame(con_mat_norm, index=class_names, columns=class_names)
    # Plotting using an heat map
    figure = plt.figure(figsize=(8,8))
    sns.set(font_scale = 2)
    sns.heatmap(con_mat_df, annot=True, cmap=plt.cm.Blues)
    plt.tight_layout()
    plt.ylabel("True label")
    plt.xlabel("Predicted label")

In [ ]:
def check_create_dir(*args):
    # Get the directory name
    temp = ""
    for i in args:
        temp = os.path.join(temp, i)
    
    # Check if exists else create them all
    if os.path.isdir(temp):
        pass
    else:
        os.makedirs(temp)

# Model Development
- Simple 1D-CNN model development
- In this the model development process is kept seperate
    - Seperate model was build for different problem domains
- This serves as a benchmark for the multi-output model

## Axis Detection

### Loading the data

In [ ]:
data_dir = os.path.join(os.getcwd(), "model_data", "axis")

# Model training data
y = []
axis = {"xaxis": 0, "yaxis": 1, "zaxis": 2, "baxis": 3, "caxis": 4}
for index, file in enumerate(os.listdir(data_dir)):
    data = np.load(os.path.join(data_dir, file))
    # X
    if index == 0:
        X = data#[:, :, 2:][:, :, np.newaxis]
    else: 
        X = np.append(X, data, axis=0)
    
    # y
    temp = [True if re.search(file[0:5] + "*", x) else False for x in axis.keys()]
    class_id = [val for i, val in zip(temp, axis.values()) if i][0]
    temp = np.repeat(class_id, data.shape[0])[:, np.newaxis]
    if index == 0:
        y = temp
    else:
        y = np.append(y, temp, axis=0)
        
    sys.stdout.write(f"The file - {file}, the class_id - {class_id}\n")
    
# Shuffling the data 
X, y = shuffle(X, y, random_state=42)

In [ ]:
classes = np.bincount(y[:, 0])
for class_id, count in enumerate(classes):
    print("Class={}, Count-{}, Percentage-{:2f}%".format(class_id, count, 100 * count/classes.sum()))

### Splitting the data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, shuffle=True)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.30, random_state=42, shuffle=True)

### Build the model

In [ ]:
# Layers of 1D-CNN model
layers = ("Conv1D", "Conv1D", "BatchNormalization", "MaxPooling1D", "Conv1D", "Conv1D", "BatchNormalization", "MaxPooling1D", 
          "Flatten", "Dense", "Dropout", "Dense", "Dense")
counts = ((128, 3), (64, 3), (None,), (2,), (32, 2), (16, 2), (None,), (2,), (None,), (256,), (0.7,), (128,), (5,))
act_fn = ("relu", "softmax")
input_shape = (X_train.shape[1], 3)

# Build the model
model = energy_model.build_1DCNN_model(layers=layers, counts=counts, act_func=act_fn, input_shape=input_shape)
model.summary()

adam = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=adam, metrics=["accuracy"])

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_loss", restore_best_weights=True, patience=30)
history = model.fit(X_train, y_train, batch_size=256, epochs=500, validation_data=(X_valid, y_valid), callbacks=[early_stopping_cb])

### Plotting the results
- The training history

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title('Model Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(['Train', 'Validation'], loc='upper left')

In [ ]:
plt.cla()
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title('Model Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(['Train', 'Validation'], loc='upper left')

- Confusion Matrix

In [ ]:
class_names = list(axis.keys())
confusion_matrix(model, X_test, y_test, class_names=class_names)

In [ ]:
score = model.evaluate(X_test, y_test)
print(f"The testing loss is {round(score[0], 4)} and the testing accuracy is {round(score[1] * 100, 2)}%")

### Save Model

In [ ]:
overlap_rate = 0.50
segment_seconds = 5
window_size = segment_seconds * 17
normalization_required = True

In [ ]:
check_create_dir("model_weights")
dir_path = os.path.join(os.getcwd(), "model_weights")
model_file_name = "axis_detection" + "_ov" + str(overlap_rate) + "_w" + str(window_size) + "_n" + str(normalization_required)

# Saving the model
model.save(os.path.join(dir_path, model_file_name), save_format="h5")

## Feed rate prediction

### Loading the data

#### Load without oversampling

In [ ]:
data_dir = os.path.join(os.getcwd(), "model_data", "feedrate")

# Model training data
# Just load the data for x-axis
file = "xaxis_feeddict_ov0.5_w85_nTrue.npy"
data = np.load(os.path.join(data_dir, file), allow_pickle=True)[()]

# Go through each feed rate in the dictionary
for index, (feed_rate, segmented_points) in enumerate(data.items()):
    
    temp = np.repeat(feed_rate, segmented_points.shape[0])[:, np.newaxis]
    if index  == 0:
        y = temp
        X = segmented_points
    else:
        y = np.append(y, temp, axis=0)
        X = np.append(X, segmented_points, axis=0)
        
# Shuffling the data 
X, y = shuffle(X, y, random_state=42)

#### Load with oversampling

In [ ]:
data_dir = os.path.join(os.getcwd(), "model_data", "feedrate")

# Model training data
# Just load the data for x-axis
file = "xaxis_feeddict_ov0.5_w85_nTrue.npy"
data = np.load(os.path.join(data_dir, file), allow_pickle=True)[()]

# Augment the data 
data_ov = data_augmentation.naive_resampler(data, 1)

# Ensure the count for all classes
for key, value in data_ov.items():
    sys.stdout.write(f"The class {key} has a shape of {value.shape[0]}\n")
    
# Go through each feed rate in the dictionary
for index, (feed_rate, segmented_points) in enumerate(data_ov.items()):
    
    temp = np.repeat(feed_rate, segmented_points.shape[0])[:, np.newaxis]
    if index  == 0:
        y = temp
        X = segmented_points
    else:
        y = np.append(y, temp, axis=0)
        X = np.append(X, segmented_points, axis=0)
        
# Shuffling the data 
X, y = shuffle(X, y, random_state=42)

### Splitting the data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, shuffle=True)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.30, random_state=42, shuffle=True)

### Build the model

In [ ]:
# Layers of 1D-CNN model
layers = ("Conv1D", "Conv1D", "BatchNormalization", "MaxPooling1D",
          "Conv1D", "Conv1D", "BatchNormalization", "MaxPooling1D", "Conv1D", "Conv1D", "BatchNormalization", "MaxPooling1D", 
          "Flatten", "Dense", "Dropout", "Dense", "Dense")
counts = ((1024, 3), (512, 3), (None,), (2,), (512, 3), (256, 3), (None,), (2,), (128, 2), (64, 2), (None,), (2,), (None,), (512,), (0.7,), (256,), (1,))
act_fn = ("elu", None)
input_shape = (X_train.shape[1], 3)

# Build the model
model = energy_model.build_1DCNN_model(layers=layers, counts=counts, act_func=act_fn, input_shape=input_shape)
model.summary()

adam = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss="mean_squared_error", optimizer=adam)

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_loss", restore_best_weights=True, patience=30)
history = model.fit(X_train, y_train, batch_size=256, epochs=500, validation_data=(X_valid, y_valid), callbacks=[early_stopping_cb])

### Plotting the results

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title('Model Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(['Train', 'Validation'], loc='upper left')

In [ ]:
model.evaluate(X_test, y_test)

- Plotting a bar chart of the feed rates

In [ ]:
feed_rates = np.unique(y_test)
labels = []
actual_feed = []
predicted_feed = []

for feed_rate in feed_rates:
    indices = np.argwhere(y_test==feed_rate)[0:, 0]
    data = X_test[indices]
    # Predict and append
    predicted_avg = model.predict(data).mean()
    # Append the data to a list
    labels.append(str(round(feed_rate)))
    actual_feed.append(feed_rate)
    predicted_feed.append(predicted_avg)
    
# Plotting the bar chart
x = np.arange(len(labels)) # the label locations
width = 0.35 # the width of the bars 

fig = plt.figure(figsize=(18, 8))
ax = fig.add_axes([0, 0, 1, 1])
rects1 = ax.bar(x - width/2, actual_feed, width, label="Actual")
rects2 = ax.bar(x + width/2, predicted_feed, width, label="Predicted")

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Feed Rates')
ax.set_title('Actual feed rate vs Predicted feed rate')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

### Save Model

In [ ]:
overlap_rate = 0.50
segment_seconds = 5
window_size = segment_seconds * 17
normalization_required = True

In [ ]:
save_path = os.path.join(os.getcwd(), "model_weights")
check_create_dir(save_path, "feed_prediction")
dir_path = os.path.join(save_path, "feed_prediction")
model_file_name = "feed_prediction" + "_ov" + str(overlap_rate) + "_w" + str(window_size) + "_n" + str(normalization_required)

# Saving the model
model.save(os.path.join(dir_path, model_file_name), save_format="h5")